In [1]:
import h5py
import pandas as pd
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Resizing, Dropout, BatchNormalization, Activation, Add, GlobalAveragePooling2D, Input, Reshape, Conv2DTranspose, Cropping2D

from keras.optimizers import Adam
import numpy as np
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from tools import define_cnn, extract_features, build_model, predict_with_model, set_seed
import warnings
warnings.filterwarnings('ignore')

c:\Users\jakub\anaconda\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Normal CNN 10-dim vector

In [5]:
# load clean gdp data, keep only year, region and real_gdp columns
ukraine = pd.read_csv("data/tabular_data_ukraine.csv")

# delete Kyiv and Kyiv_Oblast
# ukraine = ukraine[ukraine["region"] != "Kyiv"]
# ukraine = ukraine[ukraine["region"] != "Kyiv_Oblast"]

# get the data for 2021, 2022 and before 2022
ukraine_2022 = ukraine[ukraine["year"] == 2022]
ukraine = ukraine[ukraine["year"].astype(int) < 2022]
ukraine_2021 = ukraine[ukraine["year"] == 2021]
ukraine_2021.reset_index(drop=True, inplace=True)
ukraine_2022.reset_index(drop=True, inplace=True)
ukraine.reset_index(drop=True, inplace=True)

# Initialise a three dimensional array to store the images with the shape (number of images, height, width, channels)
X = np.zeros((len(ukraine), 765, 1076, 1))
y = np.zeros(len(ukraine))

# load the images
for i in range(len(ukraine)):

    # get year, region, and gdp
    year = ukraine["year"][i]
    region = ukraine["region"][i]
    gdp_value = ukraine["real_gdp"][i]

    # load the image
    file_name = f"{year}_{region}_hq.h5"
    file_path = f"data/annual_region_images/{file_name}"
    
    with h5py.File(file_path, 'r') as annual_region:
        allangle_snow_free = annual_region["AllAngle_Composite_Snow_Free"][:]

    # add the values
    y[i] = gdp_value
    X[i, :, :, 0] = allangle_snow_free

# normalise the images and gdp data
maximum_x = X.max()
X = X / maximum_x

maximum_y = y.max()
y = y / maximum_y

# get indices for observations with  year = 2021: this is the test set
test_indices = np.where(ukraine["year"].astype(int) == 2021)[0]
train_indices = np.where(ukraine["year"].astype(int) != 2021)[0]

# get the train and test sets
X_train, y_train, X_test, y_test = X[train_indices], y[train_indices], X[test_indices], y[test_indices]

# get the prediction data
X_pred = np.zeros((len(ukraine_2022), 765, 1076, 1))
for i in range(len(ukraine_2022)):

    year = ukraine_2022["year"][i]
    region = ukraine_2022["region"][i]

    file_name = f"{year}_{region}_hq.h5"
    file_path = f"data/annual_region_images/{file_name}"

    with h5py.File(file_path, 'r') as annual_region:
        allangle_snow_free = annual_region["AllAngle_Composite_Snow_Free"][:]
    
    X_pred[i, :, :, 0] = allangle_snow_free

X_pred = X_pred / maximum_x

In [6]:
# train first stage, neural network
set_seed(0)
n_features = 8
model = define_cnn(n_features = n_features, n_conv = 2, n_dense = 2)
model.compile(optimizer=Adam(), loss='mean_absolute_error', metrics=['mae'])
model.fit(X_train, y_train, epochs=50, batch_size=64, validation_split=0.2)

# extract features
ukraine_stage_2, ukraine_2021, ukraine_2022 = extract_features(model, ukraine, ukraine_2021, ukraine_2022, X_train, X_pred, X_test, n_features)
del X, y, X_train, y_train, X_test, y_test, X_pred, model

# train second stage, xgboost
year = 2021
train_data = ukraine_stage_2
test_data = ukraine_2021
prediction_data = ukraine_2022
pre_war_data = pd.concat([train_data, test_data])
selected_columns =  ["feature_" + str(i) for i in range(1, n_features+1)]
model_type = "xgboost"
log_transform = False
scale = False

param_grid_xgb = {
    'eta': [0.01, 0.1, 0.2, 0.3, 0.5],
    'gamma': [100, 1000, 10000],
    'max_depth': [4, 6, 8, 10],
    'min_child_weight': [1, 2, 5],
    'random_state': [0] 
}

mae, mpe, _, pred = build_model(train_data, test_data, selected_columns, model_type, param_grid_xgb, log_transform, scale)
gdp_change, best_params = predict_with_model(pre_war_data, prediction_data, selected_columns, model_type, param_grid_xgb, log_transform, scale)

# print results
print(mae, mpe) 
print(gdp_change)

Epoch 1/50
3/3 [==============================] - 2s 525ms/step - loss: 0.1403 - mae: 0.1403 - val_loss: 0.2258 - val_mae: 0.2258
Epoch 2/50
3/3 [==============================] - 1s 221ms/step - loss: 0.1164 - mae: 0.1164 - val_loss: 0.2363 - val_mae: 0.2363
Epoch 3/50
3/3 [==============================] - 1s 226ms/step - loss: 0.1154 - mae: 0.1154 - val_loss: 0.2202 - val_mae: 0.2202
Epoch 4/50
3/3 [==============================] - 1s 210ms/step - loss: 0.1078 - mae: 0.1078 - val_loss: 0.2092 - val_mae: 0.2092
Epoch 5/50
3/3 [==============================] - 1s 204ms/step - loss: 0.1064 - mae: 0.1064 - val_loss: 0.2080 - val_mae: 0.2080
Epoch 6/50
3/3 [==============================] - 1s 206ms/step - loss: 0.1018 - mae: 0.1018 - val_loss: 0.2142 - val_mae: 0.2142
Epoch 7/50
3/3 [==============================] - 1s 217ms/step - loss: 0.1001 - mae: 0.1001 - val_loss: 0.2078 - val_mae: 0.2078
Epoch 8/50
3/3 [==============================] - 1s 257ms/step - loss: 0.0975 - mae: 0.09